### 安装基础的依赖包

In [1]:
# capture --no-stderr
%pip install -U langgraph langsmith

# Used for this tutorial; not a requirement for LangGraph
%pip install -U langchain-openai

  Obtaining dependency information for langgraph from https://files.pythonhosted.org/packages/23/a1/d88840b3932360392a91575a972462602770232b7a98e3cd851833445a55/langgraph-0.1.11-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith from https://files.pythonhosted.org/packages/97/06/f7a49f0414796ba181a31ef0a26a9f9a3af193327cbf21a668e72d769076/langsmith-0.1.93-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.3,>=0.2.22 from https://files.pythonhosted.org/packages/f0/f0/f3f61e5a4bf201ebb1d1ff069299eab0dcb32ce03882ecc27ed475a5567d/langchain_core-0.2.23-py3-none-any.whl.metadata
  Obtaining dependency information for orjson<4.0.0,>=3.9.14 from https://files.pythonhosted.org/packages/f1/37/ced8949b7d15501e41ea3268382ed30cb2d8fd4db0705a61adc654b468bf/orjson-3.10.6-cp39-cp39-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 932.1 kB/s eta 0:00:00:--:--
  Obta

In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("API_BASE_URL")

OPENAI_API_KEY: ········
API_BASE_URL: ········


In [3]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Tutorial"

LANGSMITH_API_KEY: ········


In [4]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [5]:
from langchain_openai import ChatOpenAI

api_key = os.environ.get("OPENAI_API_KEY")
base_url = os.environ.get("API_BASE_URL")

llm = ChatOpenAI(model="gpt-4o-mini",openai_api_key=api_key, openai_api_base=base_url)


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [6]:
graph_builder.add_edge(START, "chatbot")

In [7]:
graph_builder.add_edge("chatbot", END)

In [8]:
graph = graph_builder.compile()

In [9]:

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

User: hi
Assistant: Hello! How can I assist you today?
User: 你是谁
Assistant: 我是一个人工智能助手，旨在回答问题和提供信息。有什么我可以帮助你的吗？
User: 你是什么公司做的呢？
Assistant: 我是由OpenAI开发的人工智能助手。我的目的是帮助用户回答问题、提供信息和进行对话。如果你有任何问题或需要帮助，随时可以问我！
User: q
Goodbye!
